## Парсинг сообщений из вашего личного архива VK

В архиве вашего профиля ВК можно найти папку `messages`, где каждая переписка вынесена в отдельную папку. Чтобы понять какую именно папку открывать, заглянем в `index-messages.html` и найдём нужного человека. Перейдя по ссылке, в адресной строке вы увидете что-то типа:
```
file:///home/username/.fr-GZFN5j/messages/-209008242/messages0.html
```
Здесь `-209008242` именно тот номер, который нас интересует. Находим эту папку в `messages` и переносим её в папку users.

In [11]:
from bs4 import BeautifulSoup
import bs4 as bs4
import os

USERS_DIR = os.path.join(os.getcwd(), "users")


def get_user_message_files(user_dir_name: str) -> list:
    """
    Returns a list of all html files for a given user."

    :param user_dir_name: name of the user folder
    :return: list of all files in the directory corresponding to this user in the USERS_DIR folder
    """
    messages_files = []
    user_dir = os.path.join(USERS_DIR, user_dir_name)
    for file_name in os.listdir(user_dir):
        if file_name.endswith(".html"):
            messages_files.append(os.path.join(user_dir, file_name))
    return messages_files


def parse_messages(html_code) -> str:
    """
    Extracts all text messages from an HTML document with correspondence.
    Forwarded messages, pictures, and other attachments are ignored.
    Each new paragraph in the message is saved on a separate line.

    :param html_code: a string containing the HTML code to extract messages from
    :return: a string containing the extracted messages
    """
    messages_text = ""

    soup = BeautifulSoup(html_code, 'html.parser')
    message_divs = soup.find_all('div', class_='message')

    for message_div in message_divs:
        kludges_div = message_div.find('div', class_='kludges')
        if kludges_div:
            parent_div = kludges_div.find_parent('div')
            for content in parent_div:
                if type(content) == bs4.element.NavigableString:
                    messages_text += f"{content}\n"
    return messages_text


def write_user_messages_to_file(user_dir_name, output_file):
    """
     Extracts the text of all messages for the user and writes them to the output file.

    :param user_dir_name: the ID of the user whose messages will be processed
    :param output_file: the file path where the text of all messages will be written to
    """

    messages_files = get_user_message_files(user_dir_name)
    with open(output_file, 'w', encoding='utf-8') as f:
        for file in messages_files:
            with open(os.path.join(USERS_DIR, user_dir_name, file), 'r', encoding='windows-1251') as html_file:
                messages_text = parse_messages(html_file.read())
                f.write(messages_text)

## Находим сообщения, содержащие нужные слова

Сначала используем парсер и выгрузим сообщения в переменную `messages_text`:

In [19]:
USER = "!!!ENTER USER NAME!!!"

write_user_messages_to_file(USER, "output.txt")
f = open("output.txt", "r") 
messages_text = f.read()
f.close()

Теперь можно начинать искать слова

### Способ №1

Делаем всё по умному, используем специальную библиотеку обработки естественного языка.   
Я не до конца разобрался в том, как она работает, так что если будете использовать этот метод, то рекомендую создать какую-нибудь тестовую строку и попробовать найти нужное слово в ней. После этого должно стать понятнее какие строки нужно вписать в `if stemmed_toked in (...)` и `if token.endswith(...):`. В данном примере я хотел найти все вхождения слова "люблю"

In [15]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import RussianStemmer

text = messages_text
stemmer = RussianStemmer()

tokens = word_tokenize(text, language="russian")

words_count = 0
for token in tokens:
    stemmed_token = stemmer.stem(token)

    if stemmed_token in ("люб", "любл"):

        if token.endswith("лю"):
            words_count += 1

print("Вхождений слова Люблю:", words_count)


Вхождений слова 'Люблю': 1382


### Способ №2

Берём из файла с перепиской только те строчки, в которых встречается нужное нам слово. Аномалии в результате можно будет найти глазами.

In [16]:
with open('output.txt', 'r', encoding='utf-8') as input_file, \
     open('messages.txt', 'w', encoding='utf-8') as output_file:
    for line in input_file:
        if 'люблю' in line.lower():
            output_file.write(line)
